# SciStream Fabric Example UTAH ---- SALT
### This exmaple test only the SciStream Protocol itself. According to the basic setups, this version may not need switches.
This notebook will setup a demo testbed for Scistream by utilizing Fabric resources. Will support QUIC implementation between channels. And will evaluate the performance through various streaming protocols (e.g., DASH).

Additional resources:
- [SciStream](https://scistream.github.io)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [SciStream Repo](https://github.com/scistream)
- [SciStream YouTube Presentations](https://youtu.be/IWKGjAa_TOA)
- [Topology Graph](https://drive.google.com/file/d/124xh4K15c7tUcaETXmj-JSPKcQLA_oTC/view?usp=sharing)

In [1]:
 import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Set your FABRIC PROJECT ID
os.environ['FABRIC_PROJECT_ID']='f4344c37-52e3-4d0c-a354-9311342ec109'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='cqy78_0038438951'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/fabric_bastion_key' #'id_rsa_fabric'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/work/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/work/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Basic FABRIC Slice Configuration

In [2]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

In [9]:
try:
    available_resources = fablib.get_available_resources()
    print(f"Available Resources: {available_resources}")
except Exception as e:
    print(f"Error: {e}")

Available Resources: Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
UCSD        10  320/320  2560/2560  116400/116400  635/635                 2/2                             4/4                            16/16               4/4               6/6
FIU         10  320/320  2560/2560  116400/116400  635/635                 2/2                             4/4                            16/16               4/4               6/6
MICH         6  192/192  1536/1536  60600/60600    381/381                 2/2                             2/2                            10/10               2/2               3/3
UTAH        10  214/320  2152/2560  116328/116400  630/63

## Configure Slice Parameters

This section builds the experiment slice. TO BE CHANGED, according to the code. this setup may change.

Note: Fabric did not support modifying resources after create slides. 



In [3]:
# Slice 
slice_name = 'SciStream-demo-Aug-16'

# Switches --- This version may not need any switch
# s_name = "s1" 

# switch_cores = 2
# switch_ram = 8
# switch_disk = 100

# Hosts
h1_name = "h1" # consider as Facility 1 ProdApp
h2_name = "h2" # consider as Facility 1 ProdGN
h3_name = "h3" # consider as S2UC
h4_name = "h4" # consider as Facility 2 ConsGN
h5_name = "h5" # consider as Facility 2 ConsApp

h1_ip="192.168.0.1"
h2_ip="192.168.0.2" # prodGN - command: listener-ip: 192.168.0.2 s2-port 5000
h3_ip="192.168.0.3" # S2UC json file running prod and cons on 5000 but with different ip address
h4_ip="192.168.0.4" # consGN - command: listener-ip: 192.168.0.4 s2-port 5000
h5_ip="192.168.0.5"

h1_LAN_ip="172.16.0.1"
h2_LAN_ip="172.16.0.2"

h2_WAN_ip="172.16.1.1"
h4_WAN_ip="172.16.1.2"

h4_LAN_ip="172.16.2.1"
h5_LAN_ip="172.16.2.2"

host_cores = 32
host_ram = 128
host_disk = 10
# Sites
# Default set to TACC to support IPv4 and github clone...
# Should use proxy to access github once checked as IPv6 address. Or use other git platfrom, e.g., bibucket...
site_1 = 'SALT' # 'SALT' # 'NCSA' # s'WASH' # 'MAX' # 'STAR' # 'NCSA' # 'MAX' # 'UTAH' # 'DALL' # 'TACC' # 'MAX'
site_2 = 'UTAH' # 'SALT' # 'NCSA' # s'WASH' # 'MAX' # 'STAR' # 'NCSA' # 'MAX' # 'UTAH' # 'DALL' # 'TACC' # 'MAX'

control_net_name = 'control_net' #no need for S2DS test only
prod_lan_net_name = 'prod_lan_net'
gn_net_name = 'GN_WAN_NET'
cons_lan_net_name = 'cons_lan_net'

net_pub_name = 'open_net'

# All node properties
#username = 'ubuntu'
image = 'default_ubuntu_20'
vlan = '1000'
#image_type = 'qcow2'




### Create the Slice

**This part is optional. Once the slice on, there is no need to have this block running.** \
In terms of the NIC usage. We prefer NIC_ConnectX_6 or NIC_Basic. They both provide 100 Gbps. However, NIC_ConnectX_5 still accepable. \
The network topology will be fixed and non-programmable after creating the slice. Be care of setup l2 or l3 networks. \
Network can be setup cross-site. A vlan is established based on your settings. The RTT is based on the physical distances. \
*In the newest FABRIC release, project manager may need to permissions from Admin to add cross-cite communication. Please attention.*

In [12]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Add switch node
#     switch1 = slice.add_node(name=s_name, site=site_2)
#     switch1.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
#     switch1.set_image(image)
#     #[s1_iface_local] = s1.add_component(model='NIC_Basic', name="s1_local_nic").get_interfaces()
#     [s_iface_to_h1, s_iface_to_h2] = switch1.add_component(model='NIC_ConnectX_6', name="prod_switch_nic").get_interfaces()
#     s_iface_to_h1.set_vlan(vlan=vlan)
#     s_iface_to_h2.set_vlan(vlan=vlan)

#     s_iface_to_h3 = switch1.add_component(model='NIC_Basic', name="uc_local_nic").get_interfaces()[0]
#     s_iface_to_h3.set_vlan(vlan=vlan)
#     [s_iface_to_h4, s_iface_to_h5] = switch1.add_component(model='NIC_ConnectX_6', name="cons_switch_nic").get_interfaces()
#     s_iface_to_h4.set_vlan(vlan=vlan)
#     s_iface_to_h5.set_vlan(vlan=vlan)
    
    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site_1)
    h1.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h1.set_image(image)
    [h1_iface,h1_lan_iface] = h1.add_component(model='NIC_ConnectX_6', name="h1_nic").get_interfaces()
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site_1)
    h2.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h2.set_image(image)
    [h2_iface,h2_lan_iface] = h2.add_component(model='NIC_ConnectX_6', name="h2_nic").get_interfaces()
    wan_iface1 = h2.add_component(model='NIC_Basic', name="h2_wan_nic").get_interfaces()[0]
 
    # Add host node h3 (no need for S2DS only test, but required for whole SciStream test)
    h3 = slice.add_node(name=h3_name, site=site_2)
    h3.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h3.set_image(image)
    h3_iface = h3.add_component(model='NIC_Basic', name="h3_nic").get_interfaces()[0]
    
    # Add host node h4
    h4 = slice.add_node(name=h4_name, site=site_2)
    h4.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h4.set_image(image)
    [h4_iface,h4_lan_iface] = h4.add_component(model='NIC_ConnectX_6', name="h4_nic").get_interfaces()
    wan_iface2 = h4.add_component(model='NIC_Basic', name="h4_wan_nic").get_interfaces()[0]
    
    # Add host node h5
    h5 = slice.add_node(name=h5_name, site=site_2)
    h5.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h5.set_image(image)
    [h5_iface,h5_lan_iface] = h5.add_component(model='NIC_ConnectX_6', name="h5_nic").get_interfaces()
    

    #Add control panel networks 
    control_net = slice.add_l2network(name=control_net_name, interfaces=[h1_iface,h2_iface, h3_iface, h4_iface,h5_iface])
    
    #Add Prod LAN
    prod_lan = slice.add_l2network(name=prod_lan_net_name, interfaces=[h1_lan_iface,h2_lan_iface])
    #Add Cons LAN
    cons_lan = slice.add_l2network(name=cons_lan_net_name, interfaces=[h4_lan_iface,h5_lan_iface])
    
    #Add GN WAN
    gn_wan = slice.add_l2network(name=gn_net_name, interfaces=[wan_iface1,wan_iface2])

    
    #Add L3 network for S2UC (May get error from Fabric)
    #net2 = slice.add_l3network(name=net_pub_name, interfaces=[h3_iface_pub], type='IPv4')
    
    #Submit Slice Request
    slice.submit() #(wait_progress=True)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    


---------------  ------------------------------------
Slice Name       SciStream-demo-Aug-16
Slice ID         28f2b6ac-2b2c-4191-84cd-1c73d6b8d8a2
Slice State      StableOK
Lease End (UTC)  2022-08-17 20:07:43 +0000
---------------  ------------------------------------

Retry: 16, Time: 217 sec

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image              Management IP                           State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  -----------------  --------------------------------------  -------  -------
268795a6-24d7-428a-b999-9aa71b255ce9  h1      SALT    salt-w2.fabric-testbed.net       32    128      10  default_ubuntu_20  2001:400:a100:3010:f816:3eff:fe18:4850  Active
75935c26-16f6-4d78-834c-c0bbf2b05694  h2      SALT    salt-w2.fabric-testbed.net       32    128      10  default_ubuntu_20  2001:400:a100:3010:f816:3eff:fe6f:3293  Act

# timeout needs to be extended

## Get the Slice

In [35]:
# ssh -F ~/work/fabric_config/ssh_config -i ~/work/id_rsa 
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"Node: {node.get_ssh_command()}")
except Exception as e:
    print(f"Exception: {e}")

Node: ssh -i /home/fabric/work/id_rsa -J cqy78_0038438951@bastion-1.fabric-testbed.net ubuntu@2001:400:a100:3010:f816:3eff:fe18:4850
Node: ssh -i /home/fabric/work/id_rsa -J cqy78_0038438951@bastion-1.fabric-testbed.net ubuntu@2001:400:a100:3010:f816:3eff:fe6f:3293
Node: ssh -i /home/fabric/work/id_rsa -J cqy78_0038438951@bastion-1.fabric-testbed.net ubuntu@2001:1948:417:7:f816:3eff:feda:ceb
Node: ssh -i /home/fabric/work/id_rsa -J cqy78_0038438951@bastion-1.fabric-testbed.net ubuntu@2001:1948:417:7:f816:3eff:fed7:cf47
Node: ssh -i /home/fabric/work/id_rsa -J cqy78_0038438951@bastion-1.fabric-testbed.net ubuntu@2001:1948:417:7:f816:3eff:fe02:fb74


## Configure Nodes
 ### This is just an exmple. Please wait for github to turn public and then run the script. Or the system will wait for cerdential.
 ### STEP 1: Setup controll net

In [14]:
# host_config_script_h1 = ""
# if(s1_ip_type == "IPv4"):
#     host_config_scrip_h1 = 'sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && sudo apt install -qq -y python3-pip &'
#     # && git clone https://github.com/scistream/scistream.git && cd scistream/ && pip install -r requirements.txt
# else:
#     host_config_script_h1 = 'sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://CaesarQu@bitbucket.org/caesarqu/xxx.git'

# host_config_script_h2 = ""
# if(s1_ip_type == "IPv4"):
#     host_config_scrip_h2 = 'sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && sudo apt install -qq -y python3-pip &'
#     # && git clone https://github.com/scistream/scistream.git && cd scistream/ && pip install -r requirements.txt
# else:
#     host_config_script_h2 = 'sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://CaesarQu@bitbucket.org/caesarqu/xxx.git'

# host_config_script_h3 = ""
# if(s1_ip_type == "IPv4"):
#     host_config_scrip_h3 = 'sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && sudo apt install -qq -y python3-pip &'
#     # && git clone https://github.com/scistream/scistream.git && cd scistream/ && pip install -r requirements.txt
# else:
#     host_config_script_h3 = 'sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://CaesarQu@bitbucket.org/caesarqu/xxx.git'
    
try:
    h1 = slice.get_node(name=h1_name)        
    h1_os_iface = h1.get_interface(network_name=control_net_name)
    h1_os_iface.set_ip(ip=h1_ip, cidr="24")
    
    print(f"ifname: {h1_os_iface.get_os_interface()}")
    print(f"mac: {h1_os_iface.get_mac()}")
    stdout, stderr = h1.execute(f'sudo ip addr list {h1_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h1.execute(host_config_script_h1)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h2 = slice.get_node(name=h2_name)
    h2_os_iface = h2.get_interface(network_name=control_net_name)
    h2_os_iface.set_ip(ip=h2_ip, cidr="24")
 
    print(f"ifname: {h2_os_iface.get_os_interface()}")
    print(f"mac: {h2_os_iface.get_mac()}")
    stdout, stderr = h2.execute(f'sudo ip addr list {h2_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h2.execute(host_config_script_h2)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h3 = slice.get_node(name=h3_name)
    h3_os_iface = h3.get_interface(network_name=control_net_name)
    h3_os_iface.set_ip(ip=h3_ip, cidr="24")
    
    #-----setup L3 IP address for host 3
#     h3_pub_iface = h3.get_interface(network_name=net_pub_name)
#     network_pub = slice.get_network(name=net_pub_name)
#     network_pub_available_ips = network_pub.get_available_ips()
#     print(network_pub)
#     pub_addr = network_pub_available_ips.pop(0)
#     h3_pub_iface.ip_addr_add(addr=pub_addr, subnet=network_pub.get_subnet())
    #---------*
    
    print(f"ifname: {h3_os_iface.get_os_interface()}")
    print(f"mac: {h3_os_iface.get_mac()}")
    stdout, stderr = h3.execute(f'sudo ip addr list {h3_os_iface.get_os_interface()}') # && ip addr list {h3_pub_iface.get_os_interface()}
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h4 = slice.get_node(name=h4_name)
    h4_os_iface = h4.get_interface(network_name=control_net_name)
    h4_os_iface.set_ip(ip=h4_ip, cidr="24")

    print(f"ifname: {h4_os_iface.get_os_interface()}")
    print(f"mac: {h4_os_iface.get_mac()}")
    stdout, stderr = h4.execute(f'sudo ip addr list {h4_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h5 = slice.get_node(name=h5_name)
    h5_os_iface = h5.get_interface(network_name=control_net_name)
    h5_os_iface.set_ip(ip=h5_ip, cidr="24")

    print(f"ifname: {h5_os_iface.get_os_interface()}")
    print(f"mac: {h5_os_iface.get_mac()}")
    stdout, stderr = h5.execute(f'sudo ip addr list {h5_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

ifname: ens7
mac: B8:CE:F6:5D:45:6E
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether b8:ce:f6:5d:45:6e brd ff:ff:ff:ff:ff:ff
    inet 192.168.0.1/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::bace:f6ff:fe5d:456e/64 scope link 
       valid_lft forever preferred_lft forever


ifname: ens7
mac: B8:CE:F6:5D:45:4E
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether b8:ce:f6:5d:45:4e brd ff:ff:ff:ff:ff:ff
    inet 192.168.0.2/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::bace:f6ff:fe5d:454e/64 scope link 
       valid_lft forever preferred_lft forever


ifname: ens7
mac: 02:69:29:21:73:A8
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:69:29:21:73:a8 brd ff:ff:ff:ff:ff:ff
    inet 192.168.0.3/24 scope global ens7
       valid_lft forever 

 ### STEP 2: Setup LAN net

In [15]:
try:
    h1 = slice.get_node(name=h1_name)        
    h1_os_iface = h1.get_interface(network_name=prod_lan_net_name)
    h1_os_iface.set_ip(ip=h1_LAN_ip, cidr="24")
    
    print(f"ifname: {h1_os_iface.get_os_interface()}")
    print(f"mac: {h1_os_iface.get_mac()}")
    stdout, stderr = h1.execute(f'sudo ip addr list {h1_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h1.execute(host_config_script_h1)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h2 = slice.get_node(name=h2_name)
    h2_os_iface = h2.get_interface(network_name=prod_lan_net_name)
    h2_os_iface.set_ip(ip=h2_LAN_ip, cidr="24")
 
    print(f"ifname: {h2_os_iface.get_os_interface()}")
    print(f"mac: {h2_os_iface.get_mac()}")
    stdout, stderr = h2.execute(f'sudo ip addr list {h2_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h2.execute(host_config_script_h2)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    

try:
    h4 = slice.get_node(name=h4_name)
    h4_os_iface = h4.get_interface(network_name=cons_lan_net_name)
    h4_os_iface.set_ip(ip=h4_LAN_ip, cidr="24")

    print(f"ifname: {h4_os_iface.get_os_interface()}")
    print(f"mac: {h4_os_iface.get_mac()}")
    stdout, stderr = h4.execute(f'sudo ip addr list {h4_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h5 = slice.get_node(name=h5_name)
    h5_os_iface = h5.get_interface(network_name=cons_lan_net_name)
    h5_os_iface.set_ip(ip=h5_LAN_ip, cidr="24")

    print(f"ifname: {h5_os_iface.get_os_interface()}")
    print(f"mac: {h5_os_iface.get_mac()}")
    stdout, stderr = h5.execute(f'sudo ip addr list {h5_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

ifname: ens8
mac: B8:CE:F6:5D:45:6F
4: ens8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether b8:ce:f6:5d:45:6f brd ff:ff:ff:ff:ff:ff
    inet 172.16.0.1/24 scope global ens8
       valid_lft forever preferred_lft forever
    inet6 fe80::bace:f6ff:fe5d:456f/64 scope link 
       valid_lft forever preferred_lft forever


ifname: ens8
mac: B8:CE:F6:5D:45:4F
4: ens8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether b8:ce:f6:5d:45:4f brd ff:ff:ff:ff:ff:ff
    inet 172.16.0.2/24 scope global ens8
       valid_lft forever preferred_lft forever
    inet6 fe80::bace:f6ff:fe5d:454f/64 scope link 
       valid_lft forever preferred_lft forever


ifname: ens9
mac: 04:3F:72:FE:A6:21
5: ens9: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 04:3f:72:fe:a6:21 brd ff:ff:ff:ff:ff:ff
    inet 172.16.2.1/24 scope global ens9
       valid_lft forever pre

 ### STEP 3: Setup WAN net

In [16]:
try:
    h2 = slice.get_node(name=h2_name)
    h2_os_iface = h2.get_interface(network_name=gn_net_name)
    h2_os_iface.set_ip(ip=h2_WAN_ip, cidr="24")
 
    print(f"ifname: {h2_os_iface.get_os_interface()}")
    print(f"mac: {h2_os_iface.get_mac()}")
    stdout, stderr = h2.execute(f'sudo ip addr list {h2_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h2.execute(host_config_script_h2)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    

try:
    h4 = slice.get_node(name=h4_name)
    h4_os_iface = h4.get_interface(network_name=gn_net_name)
    h4_os_iface.set_ip(ip=h4_WAN_ip, cidr="24")

    print(f"ifname: {h4_os_iface.get_os_interface()}")
    print(f"mac: {h4_os_iface.get_mac()}")
    stdout, stderr = h4.execute(f'sudo ip addr list {h4_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

ifname: ens9
mac: 02:4E:4A:E1:D9:B5
5: ens9: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:4e:4a:e1:d9:b5 brd ff:ff:ff:ff:ff:ff
    inet 172.16.1.1/24 scope global ens9
       valid_lft forever preferred_lft forever
    inet6 fe80::4e:4aff:fee1:d9b5/64 scope link 
       valid_lft forever preferred_lft forever


ifname: ens7
mac: 02:02:D6:2F:B4:A2
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:02:d6:2f:b4:a2 brd ff:ff:ff:ff:ff:ff
    inet 172.16.1.2/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::2:d6ff:fe2f:b4a2/64 scope link 
       valid_lft forever preferred_lft forever




## TCP Tunning and iperf test

In [26]:
stdout, stderr = h1.execute(f'sudo apt update && sudo apt install -y iperf iperf3 net-tools')
stdout, stderr = h2.execute(f'sudo apt update && sudo apt install -y iperf iperf3 net-tools')
stdout, stderr = h3.execute(f'sudo apt update && sudo apt install -y iperf iperf3 net-tools')
stdout, stderr = h4.execute(f'sudo apt update && sudo apt install -y iperf iperf3 net-tools')
stdout, stderr = h5.execute(f'sudo apt update && sudo apt install -y iperf iperf3 net-tools')

In [18]:
stdout, stderr = h1.execute('echo "net.core.rmem_max = 536870912\nnet.core.wmem_max = 536870912\nnet.ipv4.tcp_rmem = 4096 87380 536870912\nnet.ipv4.tcp_wmem = 4096 65536 536870912\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')
print (stdout)
print (stderr)
stdout, stderr = h2.execute('echo "net.core.rmem_max = 536870912\nnet.core.wmem_max = 536870912\nnet.ipv4.tcp_rmem = 4096 87380 536870912\nnet.ipv4.tcp_wmem = 4096 65536 536870912\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')
print (stdout)
print (stderr)
stdout, stderr = h3.execute('echo "net.core.rmem_max = 536870912\nnet.core.wmem_max = 536870912\nnet.ipv4.tcp_rmem = 4096 87380 536870912\nnet.ipv4.tcp_wmem = 4096 65536 536870912\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')
print (stdout)
print (stderr)
stdout, stderr = h4.execute('echo "net.core.rmem_max = 536870912\nnet.core.wmem_max = 536870912\nnet.ipv4.tcp_rmem = 4096 87380 536870912\nnet.ipv4.tcp_wmem = 4096 65536 536870912\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')
print (stdout)
print (stderr)
stdout, stderr = h5.execute('echo "net.core.rmem_max = 536870912\nnet.core.wmem_max = 536870912\nnet.ipv4.tcp_rmem = 4096 87380 536870912\nnet.ipv4.tcp_wmem = 4096 65536 536870912\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')
print (stdout)
print (stderr)

net.core.rmem_max = 536870912
net.core.wmem_max = 536870912
net.ipv4.tcp_rmem = 4096 87380 536870912
net.ipv4.tcp_wmem = 4096 65536 536870912
net.ipv4.tcp_congestion_control=htcp
net.ipv4.tcp_mtu_probing=1
net.core.default_qdisc = fq

net.core.rmem_max = 536870912
net.core.wmem_max = 536870912
net.ipv4.tcp_rmem = 4096 87380 536870912
net.ipv4.tcp_wmem = 4096 65536 536870912
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq


net.core.rmem_max = 536870912
net.core.wmem_max = 536870912
net.ipv4.tcp_rmem = 4096 87380 536870912
net.ipv4.tcp_wmem = 4096 65536 536870912
net.ipv4.tcp_congestion_control=htcp
net.ipv4.tcp_mtu_probing=1
net.core.default_qdisc = fq

net.core.rmem_max = 536870912
net.core.wmem_max = 536870912
net.ipv4.tcp_rmem = 4096 87380 536870912
net.ipv4.tcp_wmem = 4096 65536 536870912
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq


net.core.rmem_max = 536870912
net.core.wmem_max = 

In [27]:
#ATTENTION: be aware of the NIC id!
#NEED SUDO! 
stdout, stderr = h1.execute('sudo ifconfig ens7 mtu 8900 up && sudo tc qdisc add dev ens7 root fq maxrate 30gbit')
stdout, stderr = h1.execute('sudo ifconfig ens8 mtu 8900 up && sudo tc qdisc add dev ens8 root fq maxrate 30gbit')

stdout, stderr = h2.execute('sudo ifconfig ens7 mtu 8900 up && sudo tc qdisc add dev ens7 root fq maxrate 30gbit')
stdout, stderr = h2.execute('sudo ifconfig ens8 mtu 8900 up && sudo tc qdisc add dev ens8 root fq maxrate 30gbit')
stdout, stderr = h2.execute('sudo ifconfig ens9 mtu 8900 up && sudo tc qdisc add dev ens9 root fq maxrate 30gbit')

stdout, stderr = h3.execute('sudo ifconfig ens7 mtu 8900 up && sudo tc qdisc add dev ens7 root fq maxrate 30gbit')

stdout, stderr = h4.execute('sudo ifconfig ens7 mtu 8900 up && sudo tc qdisc add dev ens7 root fq maxrate 30gbit')
stdout, stderr = h4.execute('sudo ifconfig ens8 mtu 8900 up && sudo tc qdisc add dev ens8 root fq maxrate 30gbit')
stdout, stderr = h4.execute('sudo ifconfig ens9 mtu 8900 up && sudo tc qdisc add dev ens9 root fq maxrate 30gbit')

stdout, stderr = h5.execute('sudo ifconfig ens7 mtu 8900 up && sudo tc qdisc add dev ens7 root fq maxrate 30gbit')
stdout, stderr = h5.execute('sudo ifconfig ens8 mtu 8900 up && sudo tc qdisc add dev ens8 root fq maxrate 30gbit')


In [28]:

h2 = slice.get_node(name=h2_name) 
stdout, stderr = h2.execute('tmux new -d \'iperf -s -f K\'')
print (stdout)
print (stderr)

In [29]:
#to be changed to h4 and h5 to test.
h4 = slice.get_node(name=h4_name) 
stdout, stderr = h4.execute(f'iperf -c 172.16.1.1 -P 32 -w 999M') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.1.1, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 34] local 172.16.1.2 port 50248 connected with 172.16.1.1 port 5001
[ 27] local 172.16.1.2 port 50240 connected with 172.16.1.1 port 5001
[ 32] local 172.16.1.2 port 50244 connected with 172.16.1.1 port 5001
[ 36] local 172.16.1.2 port 50246 connected with 172.16.1.1 port 5001
[  5] local 172.16.1.2 port 50190 connected with 172.16.1.1 port 5001
[  6] local 172.16.1.2 port 50194 connected with 172.16.1.1 port 5001
[  4] local 172.16.1.2 port 50186 connected with 172.16.1.1 port 5001
[  8] local 172.16.1.2 port 50196 connected with 172.16.1.1 port 5001
[  7] local 172.16.1.2 port 50192 connected with 172.16.1.1 port 5001
[ 15] local 172.16.1.2 port 50198 connected with 172.16.1.1 port 5001
[  3] local 172.16.1.2 port 50188 connected with 172.16.1.1 port 5001
[

# Upload File and Execute

In [30]:
#upload_file(local_file_path, remote_file_path, retry=3, retry_interval=10)
# FILE 1: scistream source code

try:
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('scistream-source-code.zip',"scistream-source-code.zip", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('scistream-source-code.zip',"scistream-source-code.zip", retry=3, retry_interval=10)
    h3 = slice.get_node(name=h3_name)
    h3.upload_file('scistream-source-code.zip',"scistream-source-code.zip", retry=3, retry_interval=10)
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('scistream-source-code.zip',"scistream-source-code.zip", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('scistream-source-code.zip',"scistream-source-code.zip", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [31]:
#FILE 2: setup.sh
try:
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('setup.sh',"setup.sh", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('setup.sh',"setup.sh", retry=3, retry_interval=10)
    h3 = slice.get_node(name=h3_name)
    h3.upload_file('setup.sh',"setup.sh", retry=3, retry_interval=10)
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('setup.sh',"setup.sh", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('setup.sh',"setup.sh", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [6]:
#Optional file: QUIC proxy exmaple
try:
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('quic-tun_0.0.1_linux_amd64.tar.gz',"quic-tun_0.0.1_linux_amd64.tar.gz", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('quic-tun_0.0.1_linux_amd64.tar.gz',"quic-tun_0.0.1_linux_amd64.tar.gz", retry=3, retry_interval=10)
    h3 = slice.get_node(name=h3_name)
    h3.upload_file('quic-tun_0.0.1_linux_amd64.tar.gz',"quic-tun_0.0.1_linux_amd64.tar.gz", retry=3, retry_interval=10)
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('quic-tun_0.0.1_linux_amd64.tar.gz',"quic-tun_0.0.1_linux_amd64.tar.gz", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('quic-tun_0.0.1_linux_amd64.tar.gz',"quic-tun_0.0.1_linux_amd64.tar.gz", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [ ]:
#Optional file: QUIC tunnel example
try:
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    h3 = slice.get_node(name=h3_name)
    h3.upload_file('quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [6]:
#Optional file: quic distribuation - quic-go version...
try:
    slice = fablib.get_slice(name=slice_name)
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    h3 = slice.get_node(name=h3_name)
    h3.upload_file('quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [15]:
#FILE 3: mona demo, only upload on h1 and h5
try:
    slice = fablib.get_slice(name=slice_name)
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('mona-demo.zip',"mona-demo.zip", retry=3, retry_interval=10)
    
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('mona-demo.zip',"mona-demo.zip", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [8]:
#Optional file: tls certs for public usage. Ignore if test-only 
try:
    slice = fablib.get_slice(name=slice_name)
    h1 = slice.get_node(name=h1_name)
    h1.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
    h3 = slice.get_node(name=h3_name)
    h3.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
    h4 = slice.get_node(name=h4_name)
    h4.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [5]:
#Optional download file: dumpfile...
try:
    slice = fablib.get_slice(name=slice_name)
    h1 = slice.get_node(name=h1_name)
    h1.download_file("dumpFile/prodGn_LAN.cap",'prodGn_LAN.cap', retry=3, retry_interval=10)
    h2 = slice.get_node(name=h2_name)
    h2.download_file("dumpFile/prodGn_WAN.cap",'prodGn_WAN.cap', retry=3, retry_interval=10)
    h4 = slice.get_node(name=h4_name)
    h4.download_file("dumpFile/consGn_WAN.cap",'consGn_WAN.cap', retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.download_file("dumpFile/consGn_LAN.cap",'consGn_LAN.cap', retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.download_file("dumpFile/ssl_log.txt",'aioquic/ssl_log.txt', retry=3, retry_interval=10)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [7]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     h1 = slice.get_node(name=h1_name)
#     h1.download_directory("newcerts",'aioquic/newcerts', retry=3, retry_interval=10)
# except Exception as e:
#     print(f"Error: {e}")
#     traceback.print_exc()

In [10]:
try:
    slice = fablib.get_slice(name=slice_name)
    h3 = slice.get_node(name=h1_name)
    h3.download_file("sockmap_user.c",'sockmap_user.c', retry=3, retry_interval=10)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

Error: (invalid_grant) invalid refresh token


Traceback (most recent call last):
  File "/tmp/ipykernel_53/1391151585.py", line 2, in <module>
    slice = fablib.get_slice(name=slice_name)
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/fablib.py", line 426, in get_slice
    return fablib.get_default_fablib_manager().get_slice(name=name, slice_id=slice_id)
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/fablib.py", line 1219, in get_slice
    slices = self.get_slices(excludes=[SliceState.Dead, SliceState.Closing])
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/fablib.py", line 1178, in get_slices
    return_status, slices = self.get_slice_manager().slices(excludes=excludes)
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed/slice_manager/slice_manager.py", line 225, in slices
    self.refresh_tokens()
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed/slice_manager/slice_manager.py", line 160, in refresh_tokens
    rai

## Configure Nodes ----VERBOSE


In [16]:
master_script = "bash setup.sh"
try:
    h1 = slice.get_node(name=h1_name)        
    stdout, stderr = h1.execute(master_script)
    #print("stdout: {}".format(stdout))
    print("done h1...")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h2 = slice.get_node(name=h2_name)
    stdout, stderr = h2.execute(master_script)
    #print("stdout: {}".format(stdout))
    print("done h2...")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h3 = slice.get_node(name=h3_name) 
    stdout, stderr = h3.execute(master_script)
    #print("stdout: {}".format(stdout))
    print("done h3...")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h4 = slice.get_node(name=h4_name) 
    stdout, stderr = h4.execute(master_script)
    #print("stdout: {}".format(stdout))
    print("done h4...")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h5 = slice.get_node(name=h5_name) 
    stdout, stderr = h5.execute(master_script)
    #print("stdout: {}".format(stdout))
    print("done h5...")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

done h1...
done h2...
done h3...
done h4...
done h5...


## And below we receive them.

## Below are some optinal operations for the slice

### Extend the slice reservation time

In [21]:
import datetime

#Set end host to now plus 1 day
end_date = (datetime.datetime.utcnow() + datetime.timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

Exception: Failed to renew slice: Status.FAILURE, (504)
Reason: Gateway Time-out
HTTP response headers: HTTPHeaderDict({'Server': 'nginx/1.21.6', 'Date': 'Tue, 02 Aug 2022 21:57:13 GMT', 'Content-Type': 'text/html', 'Content-Length': '167', 'Connection': 'keep-alive'})
HTTP response body: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>nginx/1.21.6</center>
</body>
</html>




### Check New Lease End Date

In [23]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Lease End         : {slice.get_lease_end()}")
       
except Exception as e:
    print(f"Exception: {e}")

Lease End         : 2022-08-16 21:56:08 +0000


### DANGER ZONE --------*-------- Delete Slice

In [5]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()